In [7]:
import geopandas as gpd
import json
import pandas as pd

In [8]:
# import json file that converts full name of a state to two character abbreviation
with open('assets/state_codes.json', "r") as file:
    state_codes = json.load(file)

In [9]:
map_df = gpd.read_file("../data_raw/shapefiles/historicalcounties")
# rename columns and simplify map geometry (to make it run faster)
map_df.rename(columns={'NHGISNAM': 'county'}, inplace=True)
map_df.rename(columns={'STATENAM': 'state'}, inplace=True)
map_df['state_abrev'] = map_df.loc[:, 'state']
map_df.replace({"state_abrev": state_codes}, inplace=True)
map_df["geometry"] = map_df["geometry"].simplify(0.01).buffer(0)
map_df["Geo_FIPS"] = map_df["Geo_FIPS"].map(lambda x: int(str(x.lstrip("0"))))
map_df.to_csv('assets/map_df.csv')